In [1]:
import aiohttp
import asyncio
import csv
import os
import re
from bs4 import BeautifulSoup

async def fetch_page(session, page_id, failed_ids_file):
    url = f"https://simple.wikipedia.org/wiki/?curid={page_id}"
    retries = 3
    for attempt in range(retries):
        try:
            async with session.get(url) as response:
                if response.status == 200:
                    text = await response.text()
                    title, content = parse_page(text)
                    return [page_id, title, content]
                elif response.status == 429:
                    await asyncio.sleep(30)
                else:
                    print(f"Không lấy được trang {page_id}, mã lỗi: {response.status}")
                    raise Exception(f"HTTP {response.status}")
        except Exception as e:
            print(f"Lỗi khi crawl trang {page_id}: {e}")
    
    print(f"Thử lại thất bại cho trang {page_id}. Lưu ID vào file thất bại.")
    with open(failed_ids_file, mode='a', encoding='utf-8') as failed_file:
        failed_file.write(f"{page_id}\n")
    return [page_id, "Error", ""]

def parse_page(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    # Lấy tiêu đề
    title = soup.title.text if soup.title else "Không tìm thấy tiêu đề"
    # Loại bỏ chuỗi không mong muốn khỏi tiêu đề
    title = title.replace(" - Simple English Wikipedia, the free encyclopedia", "").strip()
    # Lấy nội dung chính
    content_div = soup.find('div', {'id': 'mw-content-text'})
    content = ""
    if content_div:
        paragraphs = content_div.find_all('p')
        for para in paragraphs:
            para_text = para.text.strip()
            
            # Loại bỏ các thẻ kiểu \displaystyle hoặc thẻ LaTeX khác
            para_text = re.sub(r'\\displaystyle[^\s]*', '', para_text)
            
            # Thay thế các dấu xuống dòng thừa bằng dấu cách
            para_text = re.sub(r'\s+', ' ', para_text)  # Thay thế các khoảng trắng thừa, bao gồm cả dấu xuống dòng

            # Thêm vào nội dung cuối cùng
            content += para_text + " "  # Thêm dấu cách giữa các đoạn văn bản
    else:
        content = "Không tìm thấy nội dung chính"
    
    return title, content

async def fetch_group(ids, filename, semaphore, failed_ids_file, requests_per_second=40):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for i, page_id in enumerate(ids):
            async with semaphore:
                tasks.append(fetch_page(session, page_id, failed_ids_file))
            if (i + 1) % requests_per_second == 0:
                await asyncio.sleep(1)
        results = await asyncio.gather(*tasks)
        with open(filename, mode='a', encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerows(results)
        print(f"Đã lưu {len(ids)} trang vào file '{filename}'")

async def crawl_all(ids, output_file, failed_ids_file, group_size=500, max_concurrent_requests=80):
    semaphore = asyncio.Semaphore(max_concurrent_requests)
    groups = [ids[i:i+group_size] for i in range(0, len(ids), group_size)]
    if not os.path.exists(output_file):
        with open(output_file, mode='w', encoding='utf-8', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["ID", "Title", "Content"])
    if not os.path.exists(failed_ids_file):
        open(failed_ids_file, 'w', encoding='utf-8').close()
    for i, group in enumerate(groups):
        print(f"Crawl nhóm {i+1}/{len(groups)} ({len(group)} trang)...")
        await fetch_group(group, output_file, semaphore, failed_ids_file, requests_per_second=40)
        print(f"Hoàn thành nhóm {i+1}/{len(groups)}!")
        if (i + 1) % 10 == 0:
            print("Nghỉ 30 giây để tránh lỗi 429...")
            await asyncio.sleep(60)

async def run_crawl(ids, output_file, failed_ids_file):
    await crawl_all(ids, output_file, failed_ids_file)
import nest_asyncio
nest_asyncio.apply()
if __name__ == "__main__":
    input_file = "/kaggle/input/faild-tong-hop/failed16.txt"
    output_file = "/kaggle/working/wki-faild.csv"
    failed_ids_file = "/kaggle/working/failed.txt"

    if not os.path.exists(input_file):
        print(f"Không tìm thấy file '{input_file}'!")
        exit()
    
    with open(input_file, mode='r', encoding='utf-8') as file:
        ids = [line for line in file]
    
    print(f"Đã đọc {len(ids)} ID từ file '{input_file}'.")
    # Chạy trực tiếp
    await run_crawl(ids, output_file, failed_ids_file)

Đã đọc 6298 ID từ file '/kaggle/input/faild-tong-hop/failed16.txt'.
Crawl nhóm 1/13 (500 trang)...
Không lấy được trang 18823
, mã lỗi: 404
Lỗi khi crawl trang 18823
: HTTP 404
Không lấy được trang 35534
, mã lỗi: 404
Lỗi khi crawl trang 35534
: HTTP 404
Không lấy được trang 45500
, mã lỗi: 404
Lỗi khi crawl trang 45500
: HTTP 404
Không lấy được trang 39264
, mã lỗi: 404
Lỗi khi crawl trang 39264
: HTTP 404
Không lấy được trang 18505
, mã lỗi: 404
Lỗi khi crawl trang 18505
: HTTP 404
Không lấy được trang 47006
, mã lỗi: 404
Lỗi khi crawl trang 47006
: HTTP 404
Không lấy được trang 67994
, mã lỗi: 404
Lỗi khi crawl trang 67994
: HTTP 404
Không lấy được trang 35534
, mã lỗi: 404
Lỗi khi crawl trang 35534
: HTTP 404
Không lấy được trang 18823
, mã lỗi: 404
Lỗi khi crawl trang 18823
: HTTP 404
Không lấy được trang 45500
, mã lỗi: 404
Lỗi khi crawl trang 45500
: HTTP 404
Không lấy được trang 47006
, mã lỗi: 404
Lỗi khi crawl trang 47006
: HTTP 404
Không lấy được trang 39264
, mã lỗi: 404
Lỗ